
Implementation Details
1. Define the Cartpole Environment
2. Define HyperParameters
3. Define Neural Network'
   1. 3 Layers Neural Network in which the input of the cartpole environment
   2. Output is the softmax layer of the action size 
   3. Adam Optimizer
   4. Loss Function- reduce_mean(discounted_reward of the episode*neg_loss_with_logits)
   5. Loop through the episodes
       1. Pass through the neural network and get probability ditribution.
       2. Take action depending upon the probability distribution
       3. Store all the states, rewards and actions untill the episode is done depending upon the actions given by the network.
       4. when done calculate the discounted rewards and pass the states passed by the neural network to train the network.
   6. This way the whole policy update is done as after each episode the policy is updated

## Defining the Cartpole Environment

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("CartPole-v0")
env = env.unwrapped
env.seed(1)

[1]

## Hyperparameters

In [3]:
gamma=0.95   ##Discount factor
episodes=1000  ##Episodes
actions=env.action_space.n  ##Number of actions
state_size=4  ##Number of states
learning_rate=0.01

## Neural Network

In [4]:
import tensorflow as tf

In [5]:
tf.compat.v1.disable_eager_execution()
input_state=tf.compat.v1.placeholder(tf.float32, shape=[None, state_size],name="input_state")
action_space=tf.compat.v1.placeholder(tf.int32, shape=[None, actions],name="action_space")
discounted_reward=tf.compat.v1.placeholder(tf.float32, shape=[None,],name="discounted_reward")
fc1=tf.keras.layers.Dense(10,activation='relu',name="fc1")(input_state)
fc2=tf.keras.layers.Dense(actions,activation='relu',name="fc2")(fc1)
fc3=tf.keras.layers.Dense(actions,name="fc3")(fc2)
action_output=tf.keras.layers.Dense(actions,activation='softmax',name="action_output")(fc3)
neg_loss_prob=tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = action_space)
loss=tf.math.reduce_mean(discounted_reward*neg_loss_prob)
training=tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Building Gameplay Bot for Cartpole

In [6]:
def discounted_rewards(rewards):
    cummulative_reward=0.0
    discounted_episode_rewards = np.zeros_like(rewards)
    for i in reversed(range(len(rewards))):
        cummulative_reward = cummulative_reward * gamma + rewards[i]
        discounted_episode_rewards[i] = cummulative_reward
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    return discounted_episode_rewards

In [7]:
allRewards = []
episode_states=[]
episode_rewards=[]
episode_actions=[]
total_rewards = 0
maximumRewardRecorded = 0
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    for each in range(episodes):
        episode_rewards_sum = 0
        state = env.reset()
        env.render()
        while True:
            action_probability=sess.run(action_output,feed_dict={input_state:state.reshape([1,4])})
            action = np.random.choice(range(action_probability.shape[1]), p=action_probability.ravel())
            new_state, reward, done, info=env.step(action)
            episode_states.append(state)
            episode_rewards.append(reward)
            action_ = np.zeros(actions)
            action_[action] = 1
            episode_actions.append(action_)
            if done:
                episode_rewards_sum = np.sum(episode_rewards)
                allRewards.append(episode_rewards_sum)
                total_rewards = np.sum(allRewards)
                maximumRewardRecorded = np.amax(allRewards)
                print("==========================================")
                print("Episode: ", each+1)
                print("Reward: ", episode_rewards_sum)
                print("Max reward so far: ", maximumRewardRecorded)
                discounted_episode_rewards=discounted_rewards(episode_rewards)
                loss_, _ = sess.run([loss, training], feed_dict={input_state: np.vstack(np.array(episode_states)),
                                                                 action_space: np.vstack(np.array(episode_actions)),
                                                                 discounted_reward: discounted_episode_rewards 
                                                                })
                episode_states, episode_actions, episode_rewards = [],[],[]
                break
            state = new_state
env.close()

Episode:  1
Reward:  23.0
Max reward so far:  23.0
Episode:  2
Reward:  15.0
Max reward so far:  23.0
Episode:  3
Reward:  14.0
Max reward so far:  23.0
Episode:  4
Reward:  27.0
Max reward so far:  27.0
Episode:  5
Reward:  34.0
Max reward so far:  34.0
Episode:  6
Reward:  20.0
Max reward so far:  34.0
Episode:  7
Reward:  11.0
Max reward so far:  34.0
Episode:  8
Reward:  21.0
Max reward so far:  34.0
Episode:  9
Reward:  21.0
Max reward so far:  34.0
Episode:  10
Reward:  19.0
Max reward so far:  34.0
Episode:  11
Reward:  30.0
Max reward so far:  34.0
Episode:  12
Reward:  10.0
Max reward so far:  34.0
Episode:  13
Reward:  18.0
Max reward so far:  34.0
Episode:  14
Reward:  17.0
Max reward so far:  34.0
Episode:  15
Reward:  13.0
Max reward so far:  34.0
Episode:  16
Reward:  30.0
Max reward so far:  34.0
Episode:  17
Reward:  22.0
Max reward so far:  34.0
Episode:  18
Reward:  26.0
Max reward so far:  34.0
Episode:  19
Reward:  11.0
Max reward so far:  34.0
Episode:  20
Reward: 

Reward:  13.0
Max reward so far:  48.0
Episode:  89
Reward:  22.0
Max reward so far:  48.0
Episode:  90
Reward:  13.0
Max reward so far:  48.0
Episode:  91
Reward:  51.0
Max reward so far:  51.0
Episode:  92
Reward:  12.0
Max reward so far:  51.0
Episode:  93
Reward:  14.0
Max reward so far:  51.0
Episode:  94
Reward:  30.0
Max reward so far:  51.0
Episode:  95
Reward:  16.0
Max reward so far:  51.0
Episode:  96
Reward:  18.0
Max reward so far:  51.0
Episode:  97
Reward:  28.0
Max reward so far:  51.0
Episode:  98
Reward:  22.0
Max reward so far:  51.0
Episode:  99
Reward:  12.0
Max reward so far:  51.0
Episode:  100
Reward:  21.0
Max reward so far:  51.0
Episode:  101
Reward:  28.0
Max reward so far:  51.0
Episode:  102
Reward:  12.0
Max reward so far:  51.0
Episode:  103
Reward:  32.0
Max reward so far:  51.0
Episode:  104
Reward:  53.0
Max reward so far:  53.0
Episode:  105
Reward:  11.0
Max reward so far:  53.0
Episode:  106
Reward:  68.0
Max reward so far:  68.0
Episode:  107
Rewa

Episode:  176
Reward:  43.0
Max reward so far:  77.0
Episode:  177
Reward:  27.0
Max reward so far:  77.0
Episode:  178
Reward:  14.0
Max reward so far:  77.0
Episode:  179
Reward:  60.0
Max reward so far:  77.0
Episode:  180
Reward:  14.0
Max reward so far:  77.0
Episode:  181
Reward:  16.0
Max reward so far:  77.0
Episode:  182
Reward:  12.0
Max reward so far:  77.0
Episode:  183
Reward:  15.0
Max reward so far:  77.0
Episode:  184
Reward:  71.0
Max reward so far:  77.0
Episode:  185
Reward:  12.0
Max reward so far:  77.0
Episode:  186
Reward:  60.0
Max reward so far:  77.0
Episode:  187
Reward:  14.0
Max reward so far:  77.0
Episode:  188
Reward:  23.0
Max reward so far:  77.0
Episode:  189
Reward:  18.0
Max reward so far:  77.0
Episode:  190
Reward:  69.0
Max reward so far:  77.0
Episode:  191
Reward:  62.0
Max reward so far:  77.0
Episode:  192
Reward:  56.0
Max reward so far:  77.0
Episode:  193
Reward:  55.0
Max reward so far:  77.0
Episode:  194
Reward:  36.0
Max reward so far:

Episode:  261
Reward:  33.0
Max reward so far:  193.0
Episode:  262
Reward:  38.0
Max reward so far:  193.0
Episode:  263
Reward:  32.0
Max reward so far:  193.0
Episode:  264
Reward:  65.0
Max reward so far:  193.0
Episode:  265
Reward:  58.0
Max reward so far:  193.0
Episode:  266
Reward:  167.0
Max reward so far:  193.0
Episode:  267
Reward:  93.0
Max reward so far:  193.0
Episode:  268
Reward:  33.0
Max reward so far:  193.0
Episode:  269
Reward:  42.0
Max reward so far:  193.0
Episode:  270
Reward:  32.0
Max reward so far:  193.0
Episode:  271
Reward:  147.0
Max reward so far:  193.0
Episode:  272
Reward:  40.0
Max reward so far:  193.0
Episode:  273
Reward:  87.0
Max reward so far:  193.0
Episode:  274
Reward:  243.0
Max reward so far:  243.0
Episode:  275
Reward:  132.0
Max reward so far:  243.0
Episode:  276
Reward:  52.0
Max reward so far:  243.0
Episode:  277
Reward:  18.0
Max reward so far:  243.0
Episode:  278
Reward:  127.0
Max reward so far:  243.0
Episode:  279
Reward:  

Reward:  111.0
Max reward so far:  453.0
Episode:  348
Reward:  260.0
Max reward so far:  453.0
Episode:  349
Reward:  51.0
Max reward so far:  453.0
Episode:  350
Reward:  145.0
Max reward so far:  453.0
Episode:  351
Reward:  640.0
Max reward so far:  640.0
Episode:  352
Reward:  261.0
Max reward so far:  640.0
Episode:  353
Reward:  194.0
Max reward so far:  640.0
Episode:  354
Reward:  385.0
Max reward so far:  640.0
Episode:  355
Reward:  205.0
Max reward so far:  640.0
Episode:  356
Reward:  277.0
Max reward so far:  640.0
Episode:  357
Reward:  172.0
Max reward so far:  640.0
Episode:  358
Reward:  102.0
Max reward so far:  640.0
Episode:  359
Reward:  214.0
Max reward so far:  640.0
Episode:  360
Reward:  113.0
Max reward so far:  640.0
Episode:  361
Reward:  86.0
Max reward so far:  640.0
Episode:  362
Reward:  241.0
Max reward so far:  640.0
Episode:  363
Reward:  91.0
Max reward so far:  640.0
Episode:  364
Reward:  15.0
Max reward so far:  640.0
Episode:  365
Reward:  201.0

...

Episode:  515
Reward:  2519.0
Max reward so far:  4304.0
Episode:  516
Reward:  123.0
Max reward so far:  4304.0
Episode:  517
Reward:  114.0
Max reward so far:  4304.0
Episode:  518
Reward:  127.0
Max reward so far:  4304.0
Episode:  519
Reward:  443.0
Max reward so far:  4304.0
Episode:  520
Reward:  104.0
Max reward so far:  4304.0
Episode:  521
Reward:  17.0
Max reward so far:  4304.0
Episode:  522
Reward:  30.0
Max reward so far:  4304.0
Episode:  523
Reward:  113.0
Max reward so far:  4304.0
Episode:  524
Reward:  101.0
Max reward so far:  4304.0
Episode:  525
Reward:  17.0
Max reward so far:  4304.0
Episode:  526
Reward:  111.0
Max reward so far:  4304.0
Episode:  527
Reward:  24.0
Max reward so far:  4304.0
Episode:  528
Reward:  105.0
Max reward so far:  4304.0
Episode:  529
Reward:  20.0
Max reward so far:  4304.0
Episode:  530
Reward:  111.0
Max reward so far:  4304.0
Episode:  531
Reward:  107.0
Max reward so far:  4304.0
Episode:  532
Reward:  20.0
Max reward so far:  4304

Episode:  598
Reward:  67.0
Max reward so far:  5142.0
Episode:  599
Reward:  4069.0
Max reward so far:  5142.0
Episode:  600
Reward:  1469.0
Max reward so far:  5142.0
Episode:  601
Reward:  2753.0
Max reward so far:  5142.0
Episode:  602
Reward:  1336.0
Max reward so far:  5142.0
Episode:  603
Reward:  3782.0
Max reward so far:  5142.0
Episode:  604
Reward:  3361.0
Max reward so far:  5142.0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\SirzechLucifer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-e5a664d7ab68>", line 14, in <module>
    action_probability=sess.run(action_output,feed_dict={input_state:state.reshape([1,4])})
  File "C:\Users\SirzechLucifer\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py", line 960, in run
    run_metadata_ptr)
  File "C:\Users\SirzechLucifer\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py", line 1183, in _run
    feed_dict_tensor, options, run_metadata)
  File "C:\Users\SirzechLucifer\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py", line 1361, in _do_run
    run_metadata)
  File "C:\Users\SirzechLucifer\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py", line 1367, in _do_call
    return fn(*args)
  File "C:\Users\SirzechLucifer\Anaconda

KeyboardInterrupt: 